In [ ]:
#!pip install dtreeviz.trees

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement dtreeviz.trees (from versions: none)
ERROR: No matching distribution found for dtreeviz.trees


In [ ]:
# import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import Data
data_name = "/content/drive/MyDrive/MLTeam/fake data - Sheet1.csv"
data = pd.read_csv(data_name)

In [ ]:
data.head(10)

,Have diseases,Diseases type,Bmi (17-32),Height(m),weight(kg),Age (20-80),Gender,Blood Glucose Levels(begin),Blood Glucose Levels(end),Walk dog time (hr) (0-100),Play time (hrs) (0-50),DAU/MAU rate,Total steps,Calories,Recovery
0,0,NaN,19,1.59,48.0,37,0,NaN,NaN,30,43,NaN,6796,271.84,1
1,1,NaN,18,1.64,48.4,72,0,NaN,NaN,4,31,NaN,8500,340.00,0
2,1,NaN,20,1.68,56.4,45,1,NaN,NaN,50,28,NaN,7345,293.80,1
3,0,NaN,26,1.59,65.7,66,0,NaN,NaN,80,1,NaN,5786,231.44,1
4,1,NaN,26,1.98,101.9,72,1,NaN,NaN,68,6,NaN,10822,432.88,1
5,0,NaN,25,2.00,100.0,30,1,NaN,NaN,76,8,NaN,12187,487.48,1
6,0,NaN,25,1.82,82.8,52,1,NaN,NaN,77,38,NaN,13224,528.96,0
7,1,NaN,32,1.53,74.9,68,1,NaN,NaN,35,12,NaN,11627,465.08,0
8,1,NaN,32,1.90,115.5,78,1,NaN,NaN,80,8,NaN,14547,581.88,0
9,1,NaN,26,1.76,80.5,32,1,NaN,NaN,100,46,NaN,11642,465.68,1


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Have diseases                29 non-null     int64  
 1   Diseases type                0 non-null      float64
 2   Bmi (17-32)                  29 non-null     int64  
 3   Height(m)                    29 non-null     float64
 4   weight(kg)                   29 non-null     float64
 5   Age (20-80)                  29 non-null     int64  
 6   Gender                       29 non-null     int64  
 7   Blood Glucose Levels(begin)  0 non-null      float64
 8   Blood Glucose Levels(end)    0 non-null      float64
 9   Walk dog time (hr) (0-100)   29 non-null     int64  
 10  Play time (hrs) (0-50)       29 non-null     int64  
 11  DAU/MAU rate                 0 non-null      float64
 12  Total steps                  29 non-null     int64  
 13  Calories              

In [ ]:
# Data Cleaning process
cols = ["Have diseases", "Bmi (17-32)",	"Height(m)",	"weight(kg)",	"Age (20-80)",	"Gender", "Walk dog time (hr) (0-100)",	"Play time (hrs) (0-50)",	"Total steps",	"Calories", "Recovery"]
data = data[cols]

In [ ]:
# Logistic Model
input = data.loc[:, 'Have diseases':"Recovery"]
target = data[["Recovery"]]

X, y = input, target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [ ]:
iterations_n = 1000
learning_rate = 0.01
num_feature = len(cols)

In [ ]:
class PytorchLRModel(nn.Module):
  def __init__(self, num_feature):
    super(PytorchLRModel, self).__init__()
    self.fc1 = nn.Linear(num_feature, 2)
  
  def forward(self, x):
    x = F.sigmoid(self.fc1(x))
    return x[:,1]

clf = PytorchLRModel(num_feature)
criterion = nn.BCELoss()

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(clf.parameters(), lr=learning_rate)
input = Variable(torch.Tensor(X_train.values))
target = Variable(torch.Tensor(y_train.values)).long()

clf.train()
for iter_ in range(iterations_n):
  optimizer.zero_grad()
  scores = clf(input)
  loss = criterion(scores.unsqueeze(1), target.float())
  loss.backward()
  optimizer.step()

  if iter_ % 100 == 0:
    print('Iteration {}, Loss: {}'.format(iter_, loss.data.cpu().numpy()))

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Iteration 0, Loss: 52.380950927734375
Iteration 100, Loss: 52.380950927734375
Iteration 200, Loss: 52.380950927734375
Iteration 300, Loss: 52.380950927734375
Iteration 400, Loss: 52.380950927734375
Iteration 500, Loss: 52.380950927734375
Iteration 600, Loss: 52.380950927734375
Iteration 700, Loss: 52.380950927734375
Iteration 800, Loss: 52.380950927734375
Iteration 900, Loss: 52.380950927734375


In [ ]:
clf.eval()
target = Variable(torch.Tensor(y_test.values)).long()
y_hat = clf(Variable(torch.Tensor(X_test.values)))
predictions = (y_hat >= 0.5).float()
print(predictions)
print(target.shape)
accuracy = torch.eq(predictions, target).sum().item() / target.size()[0]
print("Accuracy: ", accuracy)

tensor([0., 0., 0., 0., 0., 0., 0., 0.])
torch.Size([8, 1])
Accuracy:  5.0


In [ ]:
# Random Forest
rf = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth = 8, random_state = 18).fit(X_train, y_train)
prediction = rf.predict(X_test)
mse = mean_squared_error(y_test, prediction)
rmse = mse**.5
print(mse)
print(rmse)

0.058571500000000006
0.2420154953716807


In [ ]:
# Neural Network
class NN():
  def __init__(self, input_n, layers=[256,128], output_n, lr, iterations):
    self.input_layer = input_n
    self.output_layer = output_n
    self.fc1 = nn.Linear(self.input_layer, layers[0])
    self.fc2 = nn.Linear(layers[0], layers[1])
    self.fc3 = nn.Linear(layers[1], self.output_layer)

    self.lr = lr
    self.iterations = iterations
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.softmax(self.fc3(x), dim=1)
    return x

class NN_train_test():
  def __init__(self, input_n, layers, output_n, lr， iterations, num_epochs):
    self.model = NN(input_n=input_n, layers=layers, output_n=output_n, lr=lr, iterations=iterations)
    self.criterion = nn.CrossEntropyLoss()
    self.optimizer = torch.optim.SGD(self.model.parameters(),lr=lr)
    self.num_epochs = num_epochs

  def train(X, y):
    for epoch in range(self.num_epochs):
      correct = 0
      l = 0
      for i in range(X.shape[0]):
        data = X.iloc[i, :]
        result = y.iloc[i, :]
        prediction = self.model(data)
        loss = self.criterion(prediction, result)
        loss.backward()
        optimizer.step()

        _, predictions = prediction.max(dim=1)
        correct += torch.eq(predictions, result).sum().item()
        l += result.shape[0]
    accuracy = correct / l
    print("Epoch: {}, Accuracy: {}".format(epoch, accuracy))

  def test(X, y):
    pass

In [ ]:
# RNN
